In [41]:
from bcipy import bcipy
import numpy as np
import pylsl

# This test requires a running LSL stream with the following properties:
# - Type: "EEG"

# This test also requires a marker stream with the following properties:
# - Type: "Markers"
# - Marker Format: "^SPACE pressed$"


In [42]:
ch_map =  {'FCz': 0, 'Fz': 1, 'F3': 2, 'F7': 3, 'FC3': 4, 'T7': 5, 'C5': 6, 'C3': 7, 'C1': 8, 
        'Cz' : 9, 'CP3': 10, 'CPz': 11, 'P7': 12, 'P5': 13, 'P3': 14, 'P1': 15, 'Pz': 16, 
        'PO3': 17, 'Oz': 18, 'PO4': 19, 'P8': 20, 'P6': 21, 'P4': 22, 'P2': 23, 'CP4': 24, 
        'T8' : 25, 'C6': 26, 'C4' : 27, 'C2': 28, 'FC4': 29, 'F4': 30, 'F8': 31}

sel_chs = ('FCz',
            'Fz',
            'F3',
            'F7',
            'FC3',
            'T7',
            'C5',
            'C3',
            'C1',
            'Cz',
            'CP3',
            'CPz',
            'P7',
            'P5',
            'P3',
            'P1',
            'Pz',
            'PO3',
            'Oz',
            'PO4',
            'P8',
            'P6',
            'P4',
            'P2',
            'CP4',
            'T8',
            'C6',
            'C4',
            'C2',
            'FC4',
            'F4',
            'F8'
              )

channels = [i for i in range(3,17)]

In [43]:
# Create a session and graph
session = bcipy.Session.create()
graph = bcipy.Graph.create(session)

In [58]:
# Create an LSLInputStream using the factory method
# To create the object, we must pass the session, the stream predicate, the channels, the time offset, and the marker format
# Alternatively, we can pass the data and marker stream infos, which are pylsl.StreamInfo objects

lsl_object = bcipy.source.InputLSLStream.create_marker_coupled_data_stream(
    session, "type='EEG'",
    channels, 0, marker_fmt="^SPACE pressed$"
    )


# Create a tensor from the LSLInputStream. Since the LSL stream is volatile (not static) between trials, 
# we create the tensor from "handle", passing the stream object and the shape of the tensor
t_in = bcipy.Tensor.create_from_handle(session, (len(channels), 1), lsl_object)

# Create a second tensor from data, passing the session, the shape, and the data
t_in_2 = bcipy.Tensor.create_from_data(session, shape=t_in.shape, data=np.zeros(t_in.shape))

# Create an output tensor
t_out = bcipy.Tensor.create_virtual(session)



In [54]:
# Add an addition node to the graph, passing the graph, the input tensors, and the output tensor
Add = bcipy.kernels.AdditionKernel.add_addition_node(graph, t_in, t_in_2, t_out)

In [59]:
# Verify and initialize the graph
sts1 = graph.verify()

if sts1 != bcipy.BcipEnums.SUCCESS:
    print(sts1)
    print("Test Failed D=")

sts2 = graph.initialize()

if sts2 != bcipy.BcipEnums.SUCCESS:
    print(sts2)
    print("Test Failed D=")



In [ ]:

i = 0

sts = bcipy.BcipEnums.SUCCESS
while i < 10 and sts == bcipy.BcipEnums.SUCCESS:
    # Execute the graph. The execute method will automatically wait for the correct marker 
    sts = graph.execute()   
    print(t_out.data)
    i+=1 
